# Install Required Software with Conda

*Author: Alex Lewandowski; Alaska Satellite Facility*

Installs the conda environments needed to run the notebooks in this Jupyter Book.

- `hydrosar_rtc_era5`
- `hydrosar_water_mask`

---
## 0. Import Required Software and Set Some Paths

In [2]:
from pathlib import Path
import json
import shutil
import subprocess
import yaml

try:
    import ipywidgets as widgets
    import nb_conda_kernels
    import notebook
except ModuleNotFoundError:
    !mamba install -c conda-forge notebook ipywidgets nb_conda_kernels --yes
    import ipywidgets as widgets
    
    print('\n\nIf the `notebook package` was installed, you may need to restart Jupyter Lab \n'
    'before widgets in the notebook will display correctly.')  

    print('\n\nIf the `nb_conda_kernels` package was installed, you may need to restart Jupyter Lab before the \n'
         '`osl_opera_cslc_jbook_env` environment created with this notebook will be available to use \n'
          'with the notebooks in this Jupyter Book that require it')

**Where are you running this notebook and installing these environments?**

We ask this to properly set the `--prefix` path when creating environments in OpenSARLab.


If you are working in [OpenSARLab](https://asf.alaska.edu/asf-services-open-science-lab/), we will use `~/.local/envs`, otherwise we will use your system's conda `root_prefix` path.

In [3]:
osl_option = widgets.RadioButtons(
    options=['In OpenSARLab', 'Not in OpenSARLab'],
    description='Where are you working?',
    disabled=False,
    layout=widgets.Layout(min_width='800px')
)
display(osl_option)

RadioButtons(description='Where are you working?', layout=Layout(min_width='800px'), options=('In OpenSARLab',…

In [8]:
env_config_dir = Path.cwd().parent / 'Jupyter_Book_Environments'

in_osl = 'In' in osl_option.value

if in_osl:
    conda_prefix = Path.home() / '.local/envs'
else:
    conda_info = subprocess.run(
        ['conda', 'run', '-n', 'base', 'conda', 'info', '--json'], 
        capture_output=True, 
        text=True
    )
    info = json.loads(conda_info.stdout)
    conda_prefix = Path(f"{info['root_prefix']}/envs")
print(f'conda prefix: {conda_prefix}')

conda prefix: /home/jovyan/.local/envs


---
## 1. Build the `hydrosar_rtc_era5` Conda Environment

**Select whether to create the environment from configs with explicitly pinned package versions or from unpinned configs**

After running the following code cell:
- Select `Pinned` for maximum (but not guaranteed) stability when creating the environment
- Select `Unpinned` to attempt installing the latest package versions

In [ ]:
config_options = widgets.RadioButtons(
    options=['Pinned', 'Unpinned'],
    description='Create the environment from pinned or unpinned configs',
    disabled=False,
    layout=widgets.Layout(min_width='800px')
)
display(config_options)

In [ ]:
pinned = 'Pinned' in config_options.value
if pinned:
    yaml_name = 'hydrosar_rtc_era5_locked_env.yml'
    spec_file = 'hydrosar_rtc_era5_spec-file.txt'
    spec_file_pth = list(env_config_dir.glob(spec_file))[0]
else:
    yaml_name = 'hydrosar_rtc_era5_env.yml'

conda_yaml_pth = list(env_config_dir.glob(yaml_name))[0]
with open(conda_yaml_pth, 'r') as f:
    env = yaml.safe_load(f)
    env_name = env['name']

env_prefix = conda_prefix / env_name

print(f'environment name: {env_name}')
print(f'environment prefix: {env_prefix}')

In [ ]:
!mamba clean -p -t --yes
!pip cache purge

In [ ]:
# Create the environment
!mamba env create -f $conda_yaml_pth --prefix $env_prefix --force

# Provide access to the environment in notebooks as a Jupyter kernel named for the environment 
!{env_prefix}/bin/python -m ipykernel install --prefix {env_prefix} --name {env_name}

# Having created a well-named kernel, delete the default kernel named 'Python'
shutil.rmtree(env_prefix/'share/jupyter/kernels/python3')

---
## 2. Build the `hydrosar_water_mask` Conda Environment

**Select whether to create the environment from configs with explicitly pinned package versions or from unpinned configs**

After running the following code cell:
- Select `Pinned` for maximum (but not guaranteed) stability when creating the environment
- Select `Unpinned` to attempt installing the latest package versions

In [ ]:
config_options = widgets.RadioButtons(
    options=['Pinned', 'Unpinned'],
    description='Create the environment from pinned or unpinned configs',
    disabled=False,
    layout=widgets.Layout(min_width='800px')
)
display(config_options)

In [ ]:
pinned = 'Pinned' in config_options.value
if pinned:
    yaml_name = 'hydrosar_water_mask_locked_env.yml'
    spec_file = 'hydrosar_water_mask_spec-file.txt'
    spec_file_pth = list(env_config_dir.glob(spec_file))[0]
else:
    yaml_name = 'hydrosar_water_mask_env.yml'

conda_yaml_pth = list(env_config_dir.glob(yaml_name))[0]
with open(conda_yaml_pth, 'r') as f:
    env = yaml.safe_load(f)
    env_name = env['name']

env_prefix = conda_prefix / env_name

print(f'environment name: {env_name}')
print(f'environment prefix: {env_prefix}')

In [ ]:
!mamba clean -p -t --yes
!pip cache purge

In [ ]:
# Create the environment
!mamba env create -f $conda_yaml_pth --prefix $env_prefix --force

# Provide access to the environment in notebooks as a Jupyter kernel named for the environment 
!{env_prefix}/bin/python -m ipykernel install --prefix {env_prefix} --name {env_name}

# Having created a well-named kernel, delete the default kernel named 'Python'
shutil.rmtree(env_prefix/'share/jupyter/kernels/python3')

In [5]:
config_options = widgets.RadioButtons(
    options=['Pinned', 'Unpinned'],
    description='Create the environment from pinned or unpinned configs',
    disabled=False,
    layout=widgets.Layout(min_width='800px')
)
display(config_options)

RadioButtons(description='Create the environment from pinned or unpinned configs', layout=Layout(min_width='80…

In [9]:
pinned = 'Pinned' in config_options.value
if pinned:
    yaml_name = 'locked_env.yml'
    spec_file = '_spec-file.txt'
    spec_file_pth = list(env_config_dir.glob(spec_file))[0]
else:
    yaml_name = 'one_env.yml'

conda_yaml_pth = list(env_config_dir.glob(yaml_name))[0]
with open(conda_yaml_pth, 'r') as f:
    env = yaml.safe_load(f)
    env_name = env['name']

env_prefix = conda_prefix / env_name

print(f'environment name: {env_name}')
print(f'environment prefix: {env_prefix}')

environment name: hydrosar
environment prefix: /home/jovyan/.local/envs/hydrosar


In [10]:
!mamba clean -p -t --yes
!pip cache purge

There are no unused tarball(s) to remove.
There are no unused package(s) to remove.
Files removed: 14


In [12]:
# Create the environment
!mamba env create -f $conda_yaml_pth --prefix $env_prefix --force

# Provide access to the environment in notebooks as a Jupyter kernel named for the environment 
!{env_prefix}/bin/python -m ipykernel install --prefix {env_prefix} --name {env_name}

# Having created a well-named kernel, delete the default kernel named 'Python'
shutil.rmtree(env_prefix/'share/jupyter/kernels/python3')

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
[+] 0.0s
pkgs/main/noarch                                              No change
pkgs/main/linux-64                                            No change
pkgs/r/noarch                                                 No change
[+] 0.1s
pkgs/r/linux-64 ━━━━━━━━╸━━━━━━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1spkgs/r/linux-64                                               No change


Looking for: ['affine', 'astropy', 'cdsapi', 'ciso8601', 'contextily', 'fiona', 'gdal', 'geopandas', 'hydrosar', 'hyp3_sdk', 'ipyfilechooser', 'ipykernel', 'ipympl', 'jupyterlab', 'matplotlib', 'numpy', 'opensarlab_lib', 'pandas', 'pip', 'pyproj', 'rasterio', 'scikit-fuzzy', 'xarray']


Transaction

  Prefix: /home/jovyan/.local/envs/hydrosar

  Updating specs:

   - affine
   - astropy
   - cdsapi
   - ciso8601
   - contextily
   - fiona
   - gdal
   - geopandas
